In [45]:
import numpy as np
import queue

In [46]:
filename = "GLASS.STR"

In [47]:
f = open(filename, "r")

In [76]:
with  open(filename, "r") as f:
    for i in f:
        mol_atoms = []
        mol_name = i.split()[0]
        num_elems = int(i.split()[1])
        print(mol_name, num_elems)
        
        matrix = np.zeros((num_elems, num_elems))
        for i in range(num_elems):
            line = next(f) #Берём i линию в молекуле
            mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист
            
            it = 3 #С третьего элемента в строке начинаются связи
            while line.split()[it] != "000":
                matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                matrix[int(line.split()[it][:-1]) - 1, i] = 1
                it += 1
        print(matrix)
        print(mol_atoms)
            
            

GLA1 6
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]
['1C', '2C', '3C', '4C', '5C', '6C']
GLA2 7
[[0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]
['1C', '2C', '3C', '4C', '5C', '6C', '7C']
GLA3 6
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
['1C', '2C', '3C', '4C', '5C', '6C']
GLA4 8
[[0. 1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]
['1C', '2C', '3C', '4C', '5C', '6C', '7C', '8C']
GLA5 9
[[0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [1. 0. 0

In [122]:
mat = np.array([
 [0., 1., 0., 1., 0., 0., 0., 0.],
 [1., 0., 1., 0., 0., 0., 0., 0.],
 [0., 1., 0., 0., 1., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0., 0., 0.],
 [0., 0., 1., 0., 0., 1., 0., 0.],
 [0., 0., 0., 0., 1., 0., 1., 0.],
 [0., 0., 0., 0., 0., 1., 0., 1.],
 [0., 0., 0., 0., 0., 0., 1., 0.]
])
names = ['1C', '2C', '3C', '4C', '5C', '6C', '7C', '8C']

In [133]:
def make_chains(mat, names, k):
    
    def dfs(que, mat, v, k_now, k_max, chains, names):
        
        if k_now == k_max:
            name_in_num = list(que)
            name_in_num.sort()
            name_in_num = list(map(lambda i: names[i], name_in_num))
            reduce()
            print(name_in_num)
            return
            
        for i in range(0, mat.shape[0]):
            if mat[v, i] == 1 and not i in que:
                que.append(i)
                dfs(que, mat, i, k_now + 1, k_max, chains, names)
                del(que[k_now])
            
    chains = {}
    for i in range(k):
        que = []
        que.append(i)
        
        dfs(que, mat, i, 1, k, chains, names)
        del(que[0])
        
        
        

In [134]:
make_chains(mat, names, 4)

['1C', '2C', '3C', '5C']
['2C', '3C', '5C', '6C']
['1C', '2C', '3C', '4C']
['3C', '5C', '6C', '7C']
['1C', '2C', '3C', '4C']


In [102]:
q = []

In [103]:
q.append(23)
q.append(42)

In [92]:
q.remove()

TypeError: 'collections.deque' object is not callable

In [96]:
q.queue()

TypeError: 'collections.deque' object is not callable

In [130]:
list(map(lambda i: 1, [1,2,3]))

[1, 1, 1]